# py_agent: Advanced Features Tutorial

This notebook demonstrates **advanced py_agent capabilities** for power users and production deployments:

1. **Custom Preprocessing Strategies**: Advanced recipe engineering
2. **Ensemble Methods**: Combining multiple models for better accuracy
3. **Performance Debugging**: Diagnosing and fixing poor forecasts
4. **Grouped/Panel Modeling**: Multi-entity forecasting at scale
5. **Production Best Practices**: Deployment, monitoring, error handling

## Prerequisites

This tutorial assumes you've completed:
- **22_agent_complete_tutorial.ipynb** (basics)
- **23_agent_llm_mode_tutorial.ipynb** (LLM mode)
- **24_agent_domain_specific_examples.ipynb** (domain applications)

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import py_agent
from py_agent import ForecastAgent

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 50)

print("Setup complete!")

## Generate Complex Dataset

We'll create a challenging forecasting scenario to showcase advanced features:
- **Non-linear relationships**
- **Multiple seasonality**
- **Structural breaks** (regime changes)
- **Heteroscedasticity** (changing volatility)
- **Outliers and missing data**

In [ ]:
np.random.seed(789)
n_days = 730  # 2 years
dates = pd.date_range('2022-01-01', periods=n_days, freq='D')

# Base trend with structural break at day 400
trend_early = np.linspace(100, 150, 400)
trend_late = np.linspace(150, 220, n_days - 400)
trend = np.concatenate([trend_early, trend_late])

# Multiple seasonality
day_of_week = dates.dayofweek
day_of_year = dates.dayofyear
weekly_season = 20 * np.sin(2 * np.pi * day_of_week / 7)
yearly_season = 30 * np.sin(2 * np.pi * (day_of_year - 80) / 365)

# External variables with non-linear effects
x1 = np.random.normal(10, 2, n_days)
x2 = np.random.uniform(5, 15, n_days)
x3 = np.random.exponential(3, n_days)

# Non-linear relationships
x1_effect = 2 * x1 + 0.3 * x1**2  # Quadratic
x2_effect = 5 * np.log(x2)  # Logarithmic
x3_effect = 10 / (1 + np.exp(-0.5 * (x3 - 3)))  # Sigmoid

# Interaction effects
interaction = 0.5 * x1 * x2

# Heteroscedastic noise (increasing volatility)
volatility = np.linspace(5, 20, n_days)
noise = np.random.normal(0, volatility)

# Combine
y = trend + weekly_season + yearly_season + x1_effect + x2_effect + x3_effect + interaction + noise

# Add outliers (5% of data)
outlier_mask = np.random.choice([False, True], size=n_days, p=[0.95, 0.05])
y[outlier_mask] += np.random.choice([-1, 1], size=outlier_mask.sum()) * np.random.uniform(50, 100, outlier_mask.sum())

# Add missing data (3% of data)
missing_mask = np.random.choice([False, True], size=n_days, p=[0.97, 0.03])
y_with_missing = y.copy()
y_with_missing[missing_mask] = np.nan

# Create DataFrame
data = pd.DataFrame({
    'date': dates,
    'target': y_with_missing,
    'x1': x1,
    'x2': x2,
    'x3': x3
})

# Split
train_data = data.iloc[:int(0.75 * len(data))].copy()
test_data = data.iloc[int(0.75 * len(data)):].copy()

print(f"\n📊 COMPLEX DATASET")
print(f"  - Total: {len(data)} days")
print(f"  - Train: {len(train_data)} days")
print(f"  - Test: {len(test_data)} days")
print(f"  - Missing values: {data['target'].isna().sum()} ({data['target'].isna().mean()*100:.1f}%)")
print(f"  - Outliers (estimated): {outlier_mask.sum()} ({outlier_mask.mean()*100:.1f}%)")
print(f"  - Structural break: Day 400")

# Visualize
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Target with structural break
axes[0].plot(data['date'], data['target'], alpha=0.7, label='Target')
axes[0].axvline(dates[400], color='red', linestyle='--', linewidth=2, label='Structural Break')
axes[0].axvline(train_data['date'].iloc[-1], color='gray', linestyle='--', label='Train/Test')
axes[0].set_title('Target Variable (with missing data, outliers, structural break)', fontweight='bold', fontsize=12)
axes[0].set_ylabel('Target')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# External variables
axes[1].plot(data['date'], data['x1'], alpha=0.7, label='x1 (quadratic effect)')
axes[1].plot(data['date'], data['x2'], alpha=0.7, label='x2 (log effect)')
axes[1].plot(data['date'], data['x3'], alpha=0.7, label='x3 (sigmoid effect)')
axes[1].set_title('External Variables', fontweight='bold', fontsize=12)
axes[1].set_ylabel('Value')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Non-linear effects
axes[2].scatter(data['x1'], data['target'], alpha=0.3, s=10)
axes[2].set_title('Non-Linear Relationship: Target vs x1', fontweight='bold', fontsize=12)
axes[2].set_xlabel('x1')
axes[2].set_ylabel('Target')
axes[2].grid(True, alpha=0.3)

# Volatility over time
rolling_std = data['target'].rolling(30).std()
axes[3].plot(data['date'], rolling_std, color='purple', alpha=0.7)
axes[3].set_title('Rolling Volatility (30-day)', fontweight='bold', fontsize=12)
axes[3].set_xlabel('Date')
axes[3].set_ylabel('Std Dev')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Feature 1: Performance Debugging with Autonomous Iteration

When a simple workflow doesn't work well, use **Phase 3.5 Autonomous Iteration** to automatically try different approaches.

In [ ]:
# Initialize agent
agent_debug = ForecastAgent(verbose=True, use_rag=True)

print("\n" + "="*80)
print("PERFORMANCE DEBUGGING WITH AUTONOMOUS ITERATION")
print("="*80)

# Try to achieve RMSE < 15
best_workflow, history = agent_debug.iterate(
    data=train_data,
    request="Forecast target with complex non-linear patterns",
    target_metric='rmse',
    target_value=15.0,  # Stop when RMSE < 15
    max_iterations=5,
    test_data=test_data
)

print(f"\n✓ Iteration complete")
print(f"  Iterations run: {len(history)}")
print(f"  Best RMSE: {history[-1].performance_metrics['rmse']:.2f}")

In [ ]:
# Analyze iteration history
print("\nIteration History:")
iteration_summary = pd.DataFrame([
    {
        'iteration': i+1,
        'approach': result.approach_taken,
        'model': result.workflow.extract_spec_parsnip().model_type,
        'rmse': result.performance_metrics['rmse'],
        'mae': result.performance_metrics.get('mae', np.nan),
        'r2': result.performance_metrics.get('r_squared', np.nan),
        'issues_detected': ', '.join(result.issues_detected) if result.issues_detected else 'None'
    }
    for i, result in enumerate(history)
])

print(iteration_summary.to_string(index=False))

# Visualize improvement
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE over iterations
axes[0].plot(range(1, len(history)+1), iteration_summary['rmse'], 
             marker='o', linewidth=2, markersize=8)
axes[0].axhline(15.0, color='red', linestyle='--', label='Target RMSE')
axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_title('RMSE Improvement Over Iterations', fontweight='bold', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# R² over iterations
axes[1].plot(range(1, len(history)+1), iteration_summary['r2'],
             marker='s', linewidth=2, markersize=8, color='green')
axes[1].set_xlabel('Iteration', fontsize=12)
axes[1].set_ylabel('R²', fontsize=12)
axes[1].set_title('R² Improvement Over Iterations', fontweight='bold', fontsize=14)
axes[1].set_ylim(0, 1)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Feature 2: Multi-Model Ensemble with Diversity

Combine multiple models for better robustness and accuracy using **Phase 3.3 Multi-Model Comparison**.

In [ ]:
# Create agent for ensemble
agent_ensemble = ForecastAgent(verbose=True, use_rag=True)

print("\n" + "="*80)
print("MULTI-MODEL ENSEMBLE")
print("="*80)

# Compare 5 models and get ensemble recommendation
results_ensemble = agent_ensemble.compare_models(
    data=train_data,
    request="Forecast target with complex patterns",
    n_models=5,
    cv_strategy='time_series',
    date_column='date',
    initial='12 months',
    assess='2 months',
    skip='1 month',
    return_ensemble=True
)

print(f"\n✓ Multi-model comparison complete")
print(f"  Models compared: {len(results_ensemble['model_ids'])}")
print(f"  Ensemble recommended: {results_ensemble['ensemble_recommended']}")

In [ ]:
# View rankings
print("\nModel Rankings:")
print(results_ensemble['rankings'].head(10).to_string(index=False))

if results_ensemble['ensemble_recommended']:
    print(f"\n✓ Ensemble Recommendation:")
    print(f"  Ensemble models: {results_ensemble['ensemble_models']}")
    print(f"  Diversity score: {results_ensemble['ensemble_diversity']:.2f}")
    print(f"  Reason: {results_ensemble['ensemble_reasoning']}")

# Visualize model comparison
fig, ax = plt.subplots(figsize=(12, 6))

rankings = results_ensemble['rankings'].head(5)
models = rankings['model_id'].values
rmse_vals = rankings['mean_rmse'].values
rmse_std = rankings['std_rmse'].values

x = np.arange(len(models))
bars = ax.bar(x, rmse_vals, yerr=rmse_std, capsize=5, alpha=0.7)

# Highlight ensemble models
if results_ensemble['ensemble_recommended']:
    for i, model in enumerate(models):
        if model in results_ensemble['ensemble_models']:
            bars[i].set_color('green')
            bars[i].set_alpha(0.9)

ax.set_xticks(x)
ax.set_xticklabels(models, rotation=45, ha='right')
ax.set_ylabel('RMSE (Mean ± Std)', fontsize=12)
ax.set_title('Top 5 Models Comparison (Green = Ensemble Members)', fontweight='bold', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Feature 3: Grouped/Panel Modeling at Scale

Forecast multiple entities (stores, products, regions) efficiently with **nested modeling**.

In [ ]:
# Create multi-entity dataset (3 stores with different patterns)
np.random.seed(999)

stores = ['Store_A', 'Store_B', 'Store_C']
grouped_data = []

for i, store in enumerate(stores):
    n = 365  # 1 year per store
    dates_store = pd.date_range('2023-01-01', periods=n, freq='D')
    
    # Different patterns per store
    base = 50 + i * 20  # Different base levels
    trend = np.linspace(base, base + 30, n)
    
    # Store A: Strong weekly seasonality
    # Store B: Strong yearly seasonality
    # Store C: Mixed
    dow = dates_store.dayofweek
    doy = dates_store.dayofyear
    
    if store == 'Store_A':
        pattern = 15 * np.sin(2 * np.pi * dow / 7)
    elif store == 'Store_B':
        pattern = 20 * np.sin(2 * np.pi * (doy - 80) / 365)
    else:
        pattern = 10 * np.sin(2 * np.pi * dow / 7) + 10 * np.sin(2 * np.pi * doy / 365)
    
    sales = trend + pattern + np.random.normal(0, 5, n)
    
    store_df = pd.DataFrame({
        'date': dates_store,
        'store': store,
        'sales': sales
    })
    grouped_data.append(store_df)

grouped_df = pd.concat(grouped_data, ignore_index=True)

# Split
train_grouped = grouped_df[grouped_df['date'] < '2023-10-01'].copy()
test_grouped = grouped_df[grouped_df['date'] >= '2023-10-01'].copy()

print(f"\n🏪 GROUPED MODELING DATA")
print(f"  - Stores: {len(stores)}")
print(f"  - Total observations: {len(grouped_df)}")
print(f"  - Train: {len(train_grouped)} ({train_grouped['date'].min()} to {train_grouped['date'].max()})")
print(f"  - Test: {len(test_grouped)} ({test_grouped['date'].min()} to {test_grouped['date'].max()})")

# Visualize
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

for i, store in enumerate(stores):
    store_data = grouped_df[grouped_df['store'] == store]
    train_store = train_grouped[train_grouped['store'] == store]
    
    axes[i].plot(store_data['date'], store_data['sales'], alpha=0.7, label='Sales')
    axes[i].axvline(train_store['date'].iloc[-1], color='gray', linestyle='--', label='Train/Test')
    axes[i].set_title(f'{store} Sales', fontweight='bold', fontsize=12)
    axes[i].set_ylabel('Sales')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

axes[2].set_xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# Use grouped modeling
from py_workflows import Workflow
from py_parsnip import linear_reg

print("\n" + "="*80)
print("GROUPED/PANEL MODELING")
print("="*80)

# Create workflow
wf_grouped = Workflow().add_formula("sales ~ date").add_model(linear_reg())

# Fit nested (separate model per store)
fit_grouped = wf_grouped.fit_nested(train_grouped, group_col='store')

print(f"\n✓ Nested models fitted")
print(f"  Groups: {len(fit_grouped.group_fits)}")

# Predict on test data
predictions_grouped = fit_grouped.predict(test_grouped)

# Extract outputs
outputs_grouped, coeffs_grouped, stats_grouped = fit_grouped.extract_outputs()

print(f"\nPer-Store Performance:")
test_stats_grouped = stats_grouped[stats_grouped['split'] == 'test']
print(test_stats_grouped[['group', 'rmse', 'mae', 'r_squared']].to_string(index=False))

In [ ]:
# Visualize per-store predictions
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

test_outputs_grouped = outputs_grouped[outputs_grouped['split'] == 'test']

for i, store in enumerate(stores):
    store_outputs = test_outputs_grouped[test_outputs_grouped['group'] == store]
    store_test = test_grouped[test_grouped['store'] == store]
    
    axes[i].plot(store_test['date'].values, store_outputs['actuals'].values,
                 label='Actual', linewidth=2, alpha=0.7)
    axes[i].plot(store_test['date'].values, store_outputs['fitted'].values,
                 label='Forecast', linestyle='--', linewidth=2, alpha=0.7)
    
    store_stats = test_stats_grouped[test_stats_grouped['group'] == store]
    rmse = store_stats['rmse'].iloc[0]
    r2 = store_stats['r_squared'].iloc[0]
    
    axes[i].set_title(f'{store} Forecast (RMSE={rmse:.2f}, R²={r2:.4f})', 
                      fontweight='bold', fontsize=12)
    axes[i].set_ylabel('Sales')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

axes[2].set_xlabel('Date')
plt.tight_layout()
plt.show()

## Feature 4: Production Best Practices

### Error Handling and Graceful Degradation

In [ ]:
print("\n" + "="*80)
print("PRODUCTION ERROR HANDLING")
print("="*80)

# Example 1: Handling missing data
data_with_gaps = train_data.copy()
data_with_gaps.loc[50:60, 'target'] = np.nan  # Large gap

try:
    agent_prod = ForecastAgent(verbose=True)
    workflow_prod = agent_prod.generate_workflow(
        data=data_with_gaps,
        request="Forecast target"
    )
    print("✓ Handled missing data gracefully")
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Recommendation: Use imputation in preprocessing")

In [ ]:
# Example 2: Budget controls for LLM mode
import os

if 'ANTHROPIC_API_KEY' in os.environ:
    try:
        agent_budget = ForecastAgent(
            verbose=False,  # Reduce verbosity in production
            use_llm=True,
            budget_per_day=10.0  # Strict budget
        )
        
        # Generate multiple workflows
        for i in range(3):
            workflow = agent_budget.generate_workflow(
                data=train_data.head(100),  # Small subset for testing
                request=f"Forecast target (iteration {i+1})"
            )
            print(f"  Iteration {i+1}: Cost=${agent_budget.llm_client.total_cost:.4f}")
        
        print(f"\n✓ Budget controls working")
        print(f"  Total cost: ${agent_budget.llm_client.total_cost:.4f}")
        print(f"  Budget limit: ${agent_budget.llm_client.budget_per_day:.2f}")
    except Exception as e:
        print(f"✓ Budget exceeded: {e}")
else:
    print("⚠️  Skipping LLM budget test (no API key)")

### Performance Monitoring

In [ ]:
print("\n" + "="*80)
print("PERFORMANCE MONITORING")
print("="*80)

# Create monitoring dashboard function
def create_monitoring_dashboard(fit_workflow, test_data):
    """
    Create monitoring dashboard for production forecasts.
    """
    # Evaluate
    eval_result = fit_workflow.evaluate(test_data)
    outputs, coeffs, stats = eval_result.extract_outputs()
    
    # Get test metrics
    test_stats = stats[stats['split'] == 'test']
    
    # Dashboard data
    dashboard = {
        'timestamp': datetime.now(),
        'model_type': fit_workflow.extract_spec_parsnip().model_type,
        'n_test_samples': len(test_data),
        'rmse': test_stats['rmse'].iloc[0],
        'mae': test_stats['mae'].iloc[0],
        'mape': test_stats['mape'].iloc[0],
        'r_squared': test_stats['r_squared'].iloc[0],
        'mean_residual': test_stats['mean_residual'].iloc[0],
        'max_error': outputs[outputs['split']=='test']['residuals'].abs().max(),
        'status': 'healthy' if test_stats['r_squared'].iloc[0] > 0.7 else 'degraded'
    }
    
    return pd.DataFrame([dashboard])

# Generate monitoring data
agent_monitor = ForecastAgent(verbose=False)
workflow_monitor = agent_monitor.generate_workflow(data=train_data, request="Forecast target")
fit_monitor = workflow_monitor.fit(train_data)

dashboard = create_monitoring_dashboard(fit_monitor, test_data)
print("\nMonitoring Dashboard:")
print(dashboard.T.to_string())

# Alert logic
if dashboard['status'].iloc[0] == 'degraded':
    print("\n⚠️  ALERT: Model performance degraded!")
    print("   Action: Retrain model or investigate data quality issues")
else:
    print("\n✓ Model performance healthy")

### Reproducibility and Version Control

In [ ]:
print("\n" + "="*80)
print("REPRODUCIBILITY & VERSION CONTROL")
print("="*80)

# Extract model configuration for versioning
model_config = {
    'model_type': fit_monitor.extract_spec_parsnip().model_type,
    'model_args': fit_monitor.extract_spec_parsnip().args,
    'formula': fit_monitor.extract_formula(),
    'engine': fit_monitor.extract_spec_parsnip().engine,
    'train_start': train_data['date'].min(),
    'train_end': train_data['date'].max(),
    'n_train_samples': len(train_data),
    'features': list(train_data.columns)
}

print("\nModel Configuration (for version control):")
for key, val in model_config.items():
    print(f"  {key}: {val}")

# Save configuration to JSON
import json
config_json = json.dumps(model_config, indent=2, default=str)
print("\nJSON Configuration:")
print(config_json[:500] + "..." if len(config_json) > 500 else config_json)

## Key Takeaways

### Advanced Features Summary

1. **Autonomous Iteration (Phase 3.5)**
   - Automatically tries different approaches until target performance reached
   - Self-debugging: detects overfitting, underfitting, etc.
   - Use for complex forecasting scenarios

2. **Multi-Model Ensembles (Phase 3.3)**
   - Compare 5+ models with cross-validation
   - Automatic ensemble recommendation with diversity scoring
   - Better robustness and accuracy

3. **Grouped/Panel Modeling**
   - Forecast multiple entities efficiently
   - Separate models per group (`fit_nested()`)
   - Global models with group features (`fit_global()`)

4. **Production Best Practices**
   - Error handling and graceful degradation
   - Budget controls for LLM mode
   - Performance monitoring dashboards
   - Model configuration versioning

### Production Deployment Checklist

**Before Deployment**:
- ✅ Test on historical data
- ✅ Set up error handling
- ✅ Configure budget limits (if using LLM)
- ✅ Create monitoring dashboard
- ✅ Version control model configurations

**During Operation**:
- ✅ Monitor performance metrics
- ✅ Track API costs (if using LLM)
- ✅ Log errors and edge cases
- ✅ Retrain periodically (concept drift)

**Performance Optimization**:
- Use rule-based mode for batch processing ($0 cost)
- Use LLM mode for critical forecasts ($4-10)
- Use RAG for domain-specific recommendations
- Use iteration for complex scenarios

### Next Steps

1. **Apply to your data**: Start with simple workflows, add complexity as needed
2. **Combine features**: Use multiple phases together for best results
3. **Monitor in production**: Set up dashboards and alerts
4. **Iterate and improve**: Use feedback to refine models

### Additional Resources

- **Documentation**: `py_agent/README.md` for full API reference
- **Examples**: See other notebooks (22-24) for basics and domain applications
- **Tests**: `tests/test_agent/` for comprehensive test suite
- **Community**: GitHub issues for questions and bug reports